In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import r2_score, mean_squared_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
!pip install catboost
import catboost as cb

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed: 0')]

In [4]:
df.head()

,target,state,year,crop,area,ndvi_1,ndvi_17,ndvi_33,ndvi_49,ndvi_65,...,temp_289,temp_297,temp_305,temp_313,temp_321,temp_329,temp_337,temp_345,temp_353,temp_361
0,33.5,North Dakota,2001,soybeans,2150000,-122.538,-214.272,-213.254,-217.498,682.852,...,14364.592,13975.030,14261.334,14322.296,14163.436,13642.882,13538.665,13432.687,13243.590,12922.187
1,33.0,North Dakota,2002,soybeans,2670000,1403.378,403.376,1442.808,1988.787,1132.954,...,14113.953,13584.834,13908.839,13802.472,13913.661,13770.797,13628.559,13847.344,13017.994,13301.362
2,29.0,North Dakota,2003,soybeans,3150000,460.362,124.444,89.070,223.286,1495.452,...,14746.688,14327.990,13190.139,13560.216,13665.805,13437.337,13376.106,13091.654,13509.886,12872.187
3,23.0,North Dakota,2004,soybeans,3750000,-126.944,-96.872,-269.606,-283.778,211.704,...,14207.479,14166.294,14074.946,14033.510,13960.308,13611.019,13583.629,13662.843,13491.902,13331.316
4,36.5,North Dakota,2005,soybeans,2950000,-54.288,532.700,1071.120,1161.004,1436.910,...,14431.722,14339.406,14147.837,14211.745,13901.456,13448.038,12962.760,13565.456,13340.081,13725.257


In [5]:
# train/test split
df_train = df[~df['year'].isin([2018, 2019, 2020, 2021])]
df_test = df[df['year'].isin([2018, 2019, 2020, 2021])]

X_train = df_train.drop(columns=['target'])
y_train = df_train['target']

X_test = df_test.drop(columns=['target'])
y_test = df_test['target']

In [6]:
numeric_features = list(X_train.drop(columns=['state', 'crop']).columns)
categorical_features = ['state', 'crop']

In [7]:
# prompt: SIMILARLY AS ABOVE CREATE A PIPELINE FOR XGBOOST AND CALCULATE RMSE R2 AND ADJUSTED R2

from xgboost import XGBRegressor

# Define ColumnTransformer for preprocessing
column_trainsformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ('scaling', StandardScaler(), numeric_features)
])

# Create a pipeline for XGBoost
xgb_pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_trainsformer),
    ('xgb', XGBRegressor())
])

# Define the parameter grid for XGBoost
param_grid_xgb = {
    'xgb__n_estimators': [100, 200, 300],
    'xgb__learning_rate': [0.01, 0.1, 0.2],
    'xgb__max_depth': [3, 5, 7],
    'xgb__subsample': [0.8, 1.0],
    'xgb__colsample_bytree': [0.8, 1.0]
}

# Perform GridSearchCV for XGBoost
search_xgb = GridSearchCV(xgb_pipeline, param_grid_xgb, n_jobs=-1, scoring='neg_mean_squared_error')
search_xgb.fit(X_train, y_train)

# Get the best XGBoost model
best_xgb = search_xgb.best_estimator_

# Fit the best model and make predictions
model_xgb = best_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_test)



In [8]:
X_train.head()

,state,year,crop,area,ndvi_1,ndvi_17,ndvi_33,ndvi_49,ndvi_65,ndvi_81,...,temp_289,temp_297,temp_305,temp_313,temp_321,temp_329,temp_337,temp_345,temp_353,temp_361
0,North Dakota,2001,soybeans,2150000,-122.538,-214.272,-213.254,-217.498,682.852,1469.132,...,14364.592,13975.030,14261.334,14322.296,14163.436,13642.882,13538.665,13432.687,13243.590,12922.187
1,North Dakota,2002,soybeans,2670000,1403.378,403.376,1442.808,1988.787,1132.954,1351.850,...,14113.953,13584.834,13908.839,13802.472,13913.661,13770.797,13628.559,13847.344,13017.994,13301.362
2,North Dakota,2003,soybeans,3150000,460.362,124.444,89.070,223.286,1495.452,2098.952,...,14746.688,14327.990,13190.139,13560.216,13665.805,13437.337,13376.106,13091.654,13509.886,12872.187
3,North Dakota,2004,soybeans,3750000,-126.944,-96.872,-269.606,-283.778,211.704,2081.337,...,14207.479,14166.294,14074.946,14033.510,13960.308,13611.019,13583.629,13662.843,13491.902,13331.316
4,North Dakota,2005,soybeans,2950000,-54.288,532.700,1071.120,1161.004,1436.910,2138.303,...,14431.722,14339.406,14147.837,14211.745,13901.456,13448.038,12962.760,13565.456,13340.081,13725.257


In [9]:
# Calculate RMSE, R2, and Adjusted R2 for XGBoost
rmse_xgb = mean_squared_error(y_test, y_pred_xgb, squared=False)
r2_xgb = r2_score(y_test, y_pred_xgb)
n = len(y_test)
p = X_test.shape[1]
adjusted_r2_xgb = 1 - (1 - r2_xgb) * (n - 1) / (n - p - 1)

print("XGBoost Results:")
print(f'RMSE: {rmse_xgb}')
print(f'R2: {r2_xgb}')
print(f'Adjusted R2: {adjusted_r2_xgb}')

XGBoost Results:
RMSE: 11.061911033421845
R2: 0.9690394048361899
Adjusted R2: 0.8228365943404199


In [10]:
categorical_features = ['state', 'crop'] 
numeric_features = ['area','year', 'ndvi_1', 'ndvi_17', 'ndvi_33', 'ndvi_49', 'ndvi_65', 'ndvi_81', 'ndvi_97', 'ndvi_113',
               'ndvi_129', 'ndvi_145', 'ndvi_161', 'ndvi_177', 'ndvi_193', 'ndvi_209', 'ndvi_225', 'ndvi_241', 'ndvi_257', 'ndvi_273',
               'ndvi_289','ndvi_305', 'ndvi_321', 'ndvi_337', 'ndvi_353', 'prec_1', 'prec_32', 'prec_60', 'prec_91', 'prec_121',
               'prec_152', 'prec_182', 'prec_213', 'prec_244', 'prec_274', 'prec_305', 'prec_335', 'temp_1', 'temp_9', 'temp_17',
               'temp_25', 'temp_33', 'temp_41', 'temp_49', 'temp_57','temp_65', 'temp_73', 'temp_81', 'temp_89', 'temp_97',
               'temp_105', 'temp_113', 'temp_121', 'temp_129', 'temp_137', 'temp_145', 'temp_153', 'temp_161', 'temp_169',
               'temp_177', 'temp_185', 'temp_193', 'temp_201', 'temp_209', 'temp_217', 'temp_225', 'temp_233', 'temp_241', 'temp_249',
               'temp_257', 'temp_265', 'temp_273', 'temp_281', 'temp_289', 'temp_297','temp_305', 'temp_313', 'temp_321', 'temp_329',
               'temp_337', 'temp_345','temp_353', 'temp_361'] 
preprocesser = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ('scaling', StandardScaler(), numeric_features)
])
preprocesser.fit(X_train)

ColumnTransformer(transformers=[('ohe', OneHotEncoder(handle_unknown='ignore'),
                                 ['state', 'crop']),
                                ('scaling', StandardScaler(),
                                 ['area', 'year', 'ndvi_1', 'ndvi_17',
                                  'ndvi_33', 'ndvi_49', 'ndvi_65', 'ndvi_81',
                                  'ndvi_97', 'ndvi_113', 'ndvi_129', 'ndvi_145',
                                  'ndvi_161', 'ndvi_177', 'ndvi_193',
                                  'ndvi_209', 'ndvi_225', 'ndvi_241',
                                  'ndvi_257', 'ndvi_273', 'ndvi_289',
                                  'ndvi_305', 'ndvi_321', 'ndvi_337',
                                  'ndvi_353', 'prec_1', 'prec_32', 'prec_60',
                                  'prec_91', 'prec_121', ...])])

In [11]:
def prediction(state, year, crop, area, ndvi_1, ndvi_17, ndvi_33, ndvi_49, ndvi_65, ndvi_81, ndvi_97, ndvi_113,
               ndvi_129, ndvi_145, ndvi_161, ndvi_177, ndvi_193, ndvi_209, ndvi_225, ndvi_241, ndvi_257, ndvi_273,
               ndvi_289,ndvi_305, ndvi_321, ndvi_337, ndvi_353, prec_1, prec_32, prec_60, prec_91, prec_121,
               prec_152, prec_182, prec_213, prec_244, prec_274, prec_305, prec_335, temp_1, temp_9, temp_17,
               temp_25, temp_33, temp_41, temp_49, temp_57,temp_65, temp_73, temp_81, temp_89, temp_97,
               temp_105, temp_113, temp_121, temp_129, temp_137, temp_145, temp_153, temp_161, temp_169,
               temp_177, temp_185, temp_193, temp_201, temp_209, temp_217, temp_225, temp_233, temp_241, temp_249,
               temp_257, temp_265, temp_273, temp_281, temp_289, temp_297,temp_305, temp_313, temp_321, temp_329,
               temp_337, temp_345,temp_353, temp_361):
    features = pd.DataFrame([[state, year, crop, area, ndvi_1, ndvi_17, ndvi_33, ndvi_49, ndvi_65, ndvi_81, ndvi_97, ndvi_113,
               ndvi_129, ndvi_145, ndvi_161, ndvi_177, ndvi_193, ndvi_209, ndvi_225, ndvi_241, ndvi_257, ndvi_273,
               ndvi_289,ndvi_305, ndvi_321, ndvi_337, ndvi_353, prec_1, prec_32, prec_60, prec_91, prec_121,
               prec_152, prec_182, prec_213, prec_244, prec_274, prec_305, prec_335, temp_1, temp_9, temp_17,
               temp_25, temp_33, temp_41, temp_49, temp_57,temp_65, temp_73, temp_81, temp_89, temp_97,
               temp_105, temp_113, temp_121, temp_129, temp_137, temp_145, temp_153, temp_161, temp_169,
               temp_177, temp_185, temp_193, temp_201, temp_209, temp_217, temp_225, temp_233, temp_241, temp_249,
               temp_257, temp_265, temp_273, temp_281, temp_289, temp_297,temp_305, temp_313, temp_321, temp_329,
               temp_337, temp_345,temp_353, temp_361]], dtype = object)
    return features

In [12]:
# prompt: print values of 1 row in dataset

# Assuming you want to print the values of the first row of your DataFrame 'df'
df.iloc[0].values

array([33.5, 'North Dakota', 2001, 'soybeans', 2150000, -122.538,
       -214.272, -213.254, -217.498, 682.852, 1469.132, 2083.309,
       2474.749, 3438.685, 4530.333, 5393.437, 6638.616, 6865.773,
       5961.936, 4948.187, 4456.61, 3791.179, 3438.789, 2843.609,
       2734.946, 2641.982, 1194.712, 568.756, 0.0, 0.001, 0.0, 0.002,
       0.002, 0.004, 0.005, 0.001, 0.001, 0.001, 0.0, 0.0, 13223.386,
       13408.814, 13147.49, 13329.556, 13349.062, 12895.716, 13113.468,
       13349.892, 13631.994, 13895.59, 13827.334, 14073.836, 14277.705,
       14260.103, 14901.378, 14866.43, 15095.936, 15085.152, 15109.064,
       15105.346, 15097.076, 14917.295696153846, 15076.45, 15191.654,
       15191.268, 15069.882, 15129.83, 15230.712, 15140.402, 15320.264,
       15242.55, 14915.748, 14843.753, 14850.33, 14868.956, 14560.185,
       14364.592, 13975.03, 14261.334, 14322.296, 14163.436, 13642.882,
       13538.665, 13432.687, 13243.59, 12922.187], dtype=object)

In [13]:
result = prediction('North Dakota', 2001, 'soybeans', 2150000, -122.538,
       -214.272, -213.254, -217.498, 682.852, 1469.132, 2083.309,
       2474.749, 3438.685, 4530.333, 5393.437, 6638.616, 6865.773,
       5961.936, 4948.187, 4456.61, 3791.179, 3438.789, 2843.609,
       2734.946, 2641.982, 1194.712, 568.756, 0.0, 0.001, 0.0, 0.002,
       0.002, 0.004, 0.005, 0.001, 0.001, 0.001, 0.0, 0.0, 13223.386,
       13408.814, 13147.49, 13329.556, 13349.062, 12895.716, 13113.468,
       13349.892, 13631.994, 13895.59, 13827.334, 14073.836, 14277.705,
       14260.103, 14901.378, 14866.43, 15095.936, 15085.152, 15109.064,
       15105.346, 15097.076, 14917.295696153846, 15076.45, 15191.654,
       15191.268, 15069.882, 15129.83, 15230.712, 15140.402, 15320.264,
       15242.55, 14915.748, 14843.753, 14850.33, 14868.956, 14560.185,
       14364.592, 13975.03, 14261.334, 14322.296, 14163.436, 13642.882,
       13538.665, 13432.687, 13243.59, 12922.187)
result

,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
0,North Dakota,2001,soybeans,2150000,-122.538,-214.272,-213.254,-217.498,682.852,1469.132,...,14364.592,13975.03,14261.334,14322.296,14163.436,13642.882,13538.665,13432.687,13243.59,12922.187


In [19]:
result

,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
0,North Dakota,2001,soybeans,2150000,-122.538,-214.272,-213.254,-217.498,682.852,1469.132,...,14364.592,13975.03,14261.334,14322.296,14163.436,13642.882,13538.665,13432.687,13243.59,12922.187


In [14]:
test = ['North Dakota', 2001, 'soybeans', 2150000, -122.538,
       -214.272, -213.254, -217.498, 682.852, 1469.132, 2083.309,
       2474.749, 3438.685, 4530.333, 5393.437, 6638.616, 6865.773,
       5961.936, 4948.187, 4456.61, 3791.179, 3438.789, 2843.609,
       2734.946, 2641.982, 1194.712, 568.756, 0.0, 0.001, 0.0, 0.002,
       0.002, 0.004, 0.005, 0.001, 0.001, 0.001, 0.0, 0.0, 13223.386,
       13408.814, 13147.49, 13329.556, 13349.062, 12895.716, 13113.468,
       13349.892, 13631.994, 13895.59, 13827.334, 14073.836, 14277.705,
       14260.103, 14901.378, 14866.43, 15095.936, 15085.152, 15109.064,
       15105.346, 15097.076, 14917.295696153846, 15076.45, 15191.654,
       15191.268, 15069.882, 15129.83, 15230.712, 15140.402, 15320.264,
       15242.55, 14915.748, 14843.753, 14850.33, 14868.956, 14560.185,
       14364.592, 13975.03, 14261.334, 14322.296, 14163.436, 13642.882,
       13538.665, 13432.687, 13243.59, 12922.187]

In [15]:
len(test)

85

In [16]:
test

['North Dakota',
 2001,
 'soybeans',
 2150000,
 -122.538,
 -214.272,
 -213.254,
 -217.498,
 682.852,
 1469.132,
 2083.309,
 2474.749,
 3438.685,
 4530.333,
 5393.437,
 6638.616,
 6865.773,
 5961.936,
 4948.187,
 4456.61,
 3791.179,
 3438.789,
 2843.609,
 2734.946,
 2641.982,
 1194.712,
 568.756,
 0.0,
 0.001,
 0.0,
 0.002,
 0.002,
 0.004,
 0.005,
 0.001,
 0.001,
 0.001,
 0.0,
 0.0,
 13223.386,
 13408.814,
 13147.49,
 13329.556,
 13349.062,
 12895.716,
 13113.468,
 13349.892,
 13631.994,
 13895.59,
 13827.334,
 14073.836,
 14277.705,
 14260.103,
 14901.378,
 14866.43,
 15095.936,
 15085.152,
 15109.064,
 15105.346,
 15097.076,
 14917.295696153846,
 15076.45,
 15191.654,
 15191.268,
 15069.882,
 15129.83,
 15230.712,
 15140.402,
 15320.264,
 15242.55,
 14915.748,
 14843.753,
 14850.33,
 14868.956,
 14560.185,
 14364.592,
 13975.03,
 14261.334,
 14322.296,
 14163.436,
 13642.882,
 13538.665,
 13432.687,
 13243.59,
 12922.187]

In [17]:
test2 = pd.DataFrame([test],columns=['state', 'year', 'crop', 'area', 'ndvi_1', 'ndvi_17', 'ndvi_33',
       'ndvi_49', 'ndvi_65', 'ndvi_81', 'ndvi_97', 'ndvi_113', 'ndvi_129',
       'ndvi_145', 'ndvi_161', 'ndvi_177', 'ndvi_193', 'ndvi_209', 'ndvi_225',
       'ndvi_241', 'ndvi_257', 'ndvi_273', 'ndvi_289', 'ndvi_305', 'ndvi_321',
       'ndvi_337', 'ndvi_353', 'prec_1', 'prec_32', 'prec_60', 'prec_91',
       'prec_121', 'prec_152', 'prec_182', 'prec_213', 'prec_244', 'prec_274',
       'prec_305', 'prec_335', 'temp_1', 'temp_9', 'temp_17', 'temp_25',
       'temp_33', 'temp_41', 'temp_49', 'temp_57', 'temp_65', 'temp_73',
       'temp_81', 'temp_89', 'temp_97', 'temp_105', 'temp_113', 'temp_121',
       'temp_129', 'temp_137', 'temp_145', 'temp_153', 'temp_161', 'temp_169',
       'temp_177', 'temp_185', 'temp_193', 'temp_201', 'temp_209', 'temp_217',
       'temp_225', 'temp_233', 'temp_241', 'temp_249', 'temp_257', 'temp_265',
       'temp_273', 'temp_281', 'temp_289', 'temp_297', 'temp_305', 'temp_313',
       'temp_321', 'temp_329', 'temp_337', 'temp_345', 'temp_353', 'temp_361'])
td = preprocesser.transform(test2)

In [18]:
test2

,state,year,crop,area,ndvi_1,ndvi_17,ndvi_33,ndvi_49,ndvi_65,ndvi_81,...,temp_289,temp_297,temp_305,temp_313,temp_321,temp_329,temp_337,temp_345,temp_353,temp_361
0,North Dakota,2001,soybeans,2150000,-122.538,-214.272,-213.254,-217.498,682.852,1469.132,...,14364.592,13975.03,14261.334,14322.296,14163.436,13642.882,13538.665,13432.687,13243.59,12922.187


In [20]:
model_xgb.predict(test2)

array([30.39905], dtype=float32)

In [21]:
import pickle
pickle.dump(model_xgb, open("model_xgb.pkl","wb"))
pickle.dump(preprocesser, open("preprocesser.pkl","wb"))